Ejercicio: *Aumentación de Datos con Modelos Generativos en el Dominio del Monotributo (AFIP)*

En el año 2020, nuestro grupo de investigación participó en el Proyecto de Transferencia de Tecnología: “BillMobile: un prototipo de chatbot basado en IA para mejorar la calidad de experiencia del cliente”. Como resultado, se obtuvo una colección de preguntas relacionadas con el Régimen de Monotributo de la Administración Federal de Ingresos Públicos (AFIP). Durante el proyecto, se aplicaron diversos enfoques de aprendizaje automático, incluyendo modelos basados en Transformers.

A pesar de haberse desarrollado un estudio integral, uno de los principales desafíos fue la **cantidad limitada de ejemplos por clase en la colección de preguntas**, lo que restringió el desempeño de los modelos entrenados.

Actividad propuesta:
Diseñar una estrategia de aumentación de datos utilizando modelos generativos actuales para ampliar la colección original de preguntas. El objetivo es generar nuevas muestras  que enriquezcan el conjunto de datos en cada una de las clases definidas durante el proceso de etiquetado.

*  Formular prompts optimizados para generar nuevas preguntas sintéticas, cuidando que estén alineadas semánticamente con cada categoría.
*   Utilizar los resultados obtenidos en la Notebook anterior para ajustar y refinar dichos prompts, priorizando la calidad y relevancia de las muestras generadas.

# Parte 1: Aumentación de Datos

In [2]:
%cd aumentacion

[Errno 2] No such file or directory: 'aumentacion'
/content


In [1]:
from aumentacion import *

labels = [
    'cantidad',
    'definicion',
    'entidad',
    'manera',
    'procedimiento',
    'razon',
    'referencia',
    'si_no',
    'temporal',
    'ubicacion'
]

dataset_path = './dataset.csv'
template_path = './template.txt'
augmented_set_path = './augmented_set.csv'
classifier_path = './classifier'

llm_data = {
    'url': "http://localhost:1234/v1/chat/completions",
    'llm_name': 'mistralai/mistral-7b-instruct-v0.3'
}

ModuleNotFoundError: No module named 'aumentacion'

In [ ]:
dataset = load_dataset(dataset_path, labels)
template = get_file_content(template_path)

llm_answers = ask_llm_to_augment(labels, dataset, template, llm_data)
augmented_set = parse_llm_answers(llm_answers)
save_as_csv(augmented_set, augmented_set_path)

# Parte 2: Evaluación del Conjunto

In [ ]:
augmented_set = load_augmented_set(augmented_set_path)

predictions = ask_classifier_to_predict(classifier_path, labels, augmented_set['question'])
eval_augmented_set(augmented_set['label'], predictions)